# dependencies

In [28]:
#Import all the dependencies
import gensim
import os
import numpy as np
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
from gensim import corpora, models
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# preprocessing

In [29]:
#now create a list that contains the name of all the text file in your data #folder

docLabels = []
docLabels = [f for f in os.listdir("sampletexts/") if 
 f.endswith(".txt")]
#comp is for visualization 2
compLabel = []
compLabel = [f for f in os.listdir("testtext/") if 
 f.endswith(".txt")]

#create a list data that stores the content of all text files in order of their names in docLabels

data = []
for doc in docLabels:
    data.append(open('sampletexts/' + doc).read())
cdata = []
for doc in compLabel:
    cdata.append(open('testtext/' + doc).read())

In [30]:
tokenizer = RegexpTokenizer(r'\w+')
stopword_set = set(stopwords.words('english'))

#This function does all cleaning of data using two objects above

def nlp_clean(data):

    new_data = []
    for d in data:
        new_str = d.lower()
        dlist = tokenizer.tokenize(new_str)
        dlist = list(set(dlist).difference(stopword_set))
        new_data.append(dlist)
    return new_data

In [31]:
#collects all documents from passed list doc_list and corresponding label_list and returns an iterator over those documents
class LabeledLineSentence(object):

    def __init__(self, doc_list, labels_list):

        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):

        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.LabeledSentence(doc,    
[self.labels_list[idx]])

In [32]:
#docLabels are all the documents and data is the corresponding data
data = nlp_clean(data)
cdata = nlp_clean(cdata)
#iterator returned over all documents
it = LabeledLineSentence(data, docLabels)
cit = LabeledLineSentence(cdata, compLabel)

In [33]:
compLabel

['jefferson-notes-on-virginia.txt']

In [34]:
len(docLabels)

24

In [35]:
totalArgs=0
for i in data:
    totalArgs=totalArgs+1

In [36]:
npdata=np.array(data)

In [37]:
npdata[1]

['compared',
 'also',
 'occurs',
 'external',
 'regularity',
 'kineseis',
 'requires',
 'times',
 'resembles',
 'explains',
 'beginning',
 'implies',
 'difference',
 'dealing',
 'eye',
 'remember',
 'lapse',
 'perceiving',
 'stated',
 'initio',
 'scientific',
 'thrown',
 'infants',
 'presentation',
 'else',
 'call',
 'continuous',
 'concurrent',
 'seeing',
 'even',
 'based',
 'frequency',
 'concerned',
 'attempts',
 'path',
 'animals',
 'find',
 'well',
 'consciousness',
 'happens',
 'within',
 'weight',
 'besides',
 'unconsciously',
 'established',
 'whereas',
 'easy',
 'recovers',
 'includes',
 'actualization',
 'purpose',
 'antecedent',
 'use',
 'acquired',
 'remembers',
 'place',
 'otherwise',
 'originally',
 'involved',
 'shall',
 'frequent',
 'introductory',
 'method',
 'towards',
 'vigorous',
 'day',
 'sayings',
 'incidental',
 'truly',
 'work',
 'finally',
 'experiencing',
 'secondly',
 'strenuous',
 'frayed',
 'concurrently',
 'comprise',
 'obliviscence',
 'contrary',
 'though

# Doc2Vec Training and save the Model

In [38]:
d2vSize = 300
model = gensim.models.Doc2Vec(size=d2vSize, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(it)

#training of model
for epoch in range(100):
 print ('iteration ' + str(epoch+1))
 model.train(it, total_examples = totalArgs, epochs=model.epochs)
 model.alpha -= 0.002
 model.min_alpha = model.alpha

#saving the created model

model.save('doc2vec.model')
print ("model saved")

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning:

The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning:

Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).



iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

# Visualization 1: vector representations into PCA

In [92]:
#Things I need to be passed from the back end
#the d2v Model
#the labels of the data
#the size used to train the model
#Categories to color them differently



d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec.model')
#put vector representations into an array
vecArray = np.zeros(shape=(len(docLabels),d2vSize))
for idx, labels in enumerate(docLabels):
    vecArray[idx]=d2v_model.docvecs[labels]
#run through PCA
d2vPca = PCA().fit_transform(vecArray)

In [40]:
print(docLabels)

['aristotle-on-263.txt', 'aristotle-on-264.txt', 'aristotle-on-265.txt', 'aristotle-on-266.txt', 'aristotle-on-267.txt', 'aristotle-on-268.txt', 'aristotle-on-269.txt', 'aristotle-on-270.txt', 'aristotle-on-271.txt', 'aristotle-on-272.txt', 'aristotle-on-273.txt', 'aristotle-on-274.txt', 'aristotle-on-275.txt', 'dickens-christmas-125.txt', 'dickens-holiday-623.txt', 'dickens-hunted-624.txt', 'plato-euthyphro-342.txt', 'plato-laws-346.txt', 'plato-philebus-352.txt', 'shakespeare-lovers-62.txt', 'shakespeare-merchant-5.txt', 'shakespeare-tempest-4.txt', 'shakespeare-two-18.txt', 'twain_tom_sawyer.txt']


In [41]:
#visualize

trace1 = go.Scatter(
    x = d2vPca[:, 0],
    y = d2vPca[:, 1],
    mode='markers',
    text = docLabels,
    hoverinfo = 'text'
)
data = [trace1]

iplot(data, filename='d2vScatter')

# Visualization 2: take any other document and find the 5 most similar from our corpus

In [83]:
#Things I need from the back end
#A document preprocessed and cleaned in the same way as the corpus
#The label of that document
#The d2v model


#infer a vector of the new document
newVec=d2v_model.infer_vector(cdata[0])
#find the most similar 10 documents
mostSim = d2v_model.docvecs.most_similar([newVec],topn=10)
#convert to numpy Arr
simArr= np.array(mostSim)
#make each number represent the percent similarity instead difference betweein 0-1
for idx,each in enumerate(simArr):
    simArr[idx,1]= 100-abs(float(simArr[idx,1]))*100

In [90]:
#graph it
data = [go.Bar(
            x=simArr[:,0],
            y=simArr[:,1]
    )]
layout = go.Layout(
    title=('Most Similar Documents to ' + compLabel[0])
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='mostSim')
    
    

# examples of what info can be pulled from doc2vec

In [ ]:
#start testing
#printing the vector of document at index 1 in docLabels
#docvec = d2v_model.docvecs[1]
#print (docvec)
#printing the vector of the file using its name
docvec = d2v_model.docvecs['aristotle-on-275.txt'] #if string tag used in training
print (docvec.size)
print (docvec)

#to get most similar document with similarity scores using document-index
similar_doc = d2v_model.docvecs.most_similar(6) 
#print (similar_doc)

#to get most similar document with similarity scores using document- name
sims = d2v_model.docvecs.most_similar('aristotle-on-275.txt')
print (sims)

#to get vector of document that are not present in corpus 
#docvec = d2v_model.docvecs.infer_vector('war.txt')
#print docvec

nomatch = d2v_model.docvecs.doesnt_match(docLabels)
print (nomatch)